In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Dropout, LSTM, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.utils import to_categorical

from cv2 import imread, resize

In [13]:
df = pd.read_csv('../input/fetal-health-classification/fetal_health.csv')

print(df.shape)
df.head()

In [21]:
columns = dict(zip(list(range(df.shape[1])), df.columns.values.tolist()))
fetal_health = df['fetal_health']

df = pd.DataFrame(StandardScaler().fit_transform(df)).rename(columns = columns)
df['fetal_health'] = fetal_health
df = df.dropna()

X = df.drop(['fetal_health'], axis=1).values
y = df['fetal_health']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [22]:
def build_():
    model = Sequential()
    
    model.add(Dense(256, input_dim = 21))
    model.add(Activation('relu'))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dense(16))
    model.add(Activation('relu'))
    model.add(Dense(3, activation = 'sigmoid'))
    
    model.compile(
        loss = 'categorical_crossentropy',
        optimizer = 'sgd',
        metrics = [keras.metrics.Recall(name = 'recall')]
    )
    return model


class MLP():
    def __init__(self, epochs, batch_size):
        self.clf = KerasClassifier(build_, epochs = epochs, batch_size = batch_size)
        
    def fit_(self, x, y):
        self.clf.fit(x, y, verbose = 0)
    
    def predict_(self, x, y):
        self.p = self.clf.predict(x)
        return np.sum(y == self.p) / len(y)
        
        
clf = MLP(epochs = 250, batch_size = 32)
clf.fit_(X_train, y_train)

clf.predict_(X_test, y_test)

In [5]:
path = '../input/intel-image-classification/seg_train/seg_train'
classes = np.array(os.listdir(path))
for i, j in enumerate(classes):
    print(i, j)

In [6]:
X = []
Y = []

for i, j in enumerate(classes):
    folder = path + '/' + str(j)
    for image in os.listdir(folder):
        img = imread((os.path.join(folder, image)), 1)[...,::-1] / 255.0
        img = resize(img, (100, 100))
        X.append(img)
        Y.append(i)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 13)

X_train = np.array(X_train)
Y_train = to_categorical(np.array(Y_train))
X_test = np.array(X_test)

In [9]:
def build_():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), input_shape = (100, 100, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    
    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation = 'softmax'))

    model.compile(
            loss = 'categorical_crossentropy',
            optimizer = 'rmsprop',
            metrics = [keras.metrics.Recall(name = 'recall'), 
                       keras.metrics.Accuracy(name = 'accuracy')]
        )

    return model


class CNN():
    def __init__(self, epochs, batch_size):
        self.clf = KerasClassifier(build_, epochs = epochs, batch_size = batch_size)
        
    def fit_(self, x, y):
        self.clf.fit(x, y, verbose = 0)
    
    def predict_(self, x, y):
        self.p = self.clf.predict(x)
        return np.sum(y == self.p) / len(y)
        
        
clf = CNN(epochs = 25, batch_size = 16)
clf.fit_(X_train, Y_train)

clf.predict_(X_test, Y_test)

In [5]:
df = pd.read_csv('../input/microsoft-stock-time-series-analysis/Microsoft_Stock.csv')
df.head()

In [6]:
df = df.drop(['Volume'], axis = 1)
df.Date = pd.to_datetime(df['Date'])
df.set_index('Date', inplace = True)

df

In [7]:
df.isnull().sum()

In [8]:
df[['High','Low','Open','Close']].plot(figsize = (16, 6))

In [9]:
sc = MinMaxScaler(feature_range=(0,1))

def load_data(datasetname, column, seq_len):
    # A support function to help prepare datasets for an RNN/LSTM/GRU
    data = datasetname.loc[:,column]

    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    
    result = np.array(result)

    #Last 10% is used for validation test, first 90% for training
    row = round(0.9 * result.shape[0])
    train = result[:int(row), :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  

    return [x_train, y_train, x_test, y_test]

In [10]:
X_train, Y_train, X_test, Y_test = load_data(df, 'Close', 50)

df['Close'][:int(len(df['Close']) * .9)].plot(figsize=(16,4),legend=True)
df['Close'][int(len(df['Close']) * .9):].plot(figsize=(16,4),legend=True)
plt.legend(['Training set (First 90%)','Test set (Last 10%)'])
plt.show()

In [11]:
model=Sequential()

model.add(LSTM(25, activation = 'relu', input_shape = (X_train.shape[1],1)))
model.add(Dense(10))
model.add(Dense(1))

model.compile(
    loss = 'mean_squared_error',
    optimizer = 'adam'
)


model.fit(X_train, Y_train, epochs=10, batch_size=32)

In [12]:
prediction = model.predict(X_test)


plt.plot(Y_test, label='Test Data') 
plt.plot(prediction, label='Prediction') 
plt.legend()
plt.show()